In [274]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [275]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from pprint import pprint

In [276]:
def retrieveData(p):
    data = open(p)
    md = BeautifulSoup(data) # md - machines data
    
    res = []
    for m in md.findAll("machine"): #year of production
        a = [] 
        
        for k in m.findAll("summary"): #age
            b = {"r": int(k["profit"]), "u": int(k["servisecost"]), "c": int(k["delta"])}
            a.append(b)
        res.append(a)
#     pprint(res)
    return res

In [277]:
def findMax_t(data):
    max_t = len(data[0])       
    return max_t


In [278]:
def retrieveDataValues(val_name, data, n, t_max):
    
    res = [[-1 for _ in range(t_max)] for _ in range(n + 1)]
    for i in range(n + 1):
        res[i][i + 2] = data[0][i+2][val_name]
        
        for t in range(i):
            res[i][t] = data[i-t][t][val_name]
        
#     pprint(res)
    return res

In [279]:
def print_table(arr, rows_title, cols_title, attr = False):
    if not attr:
        df = pd.DataFrame(r_arr, index=rows_title, columns = cols_title)
    print(df)


In [280]:
def add_color_lable(w):

    return pd.DataFrame([[f'background-color: {color(v)}' if v else '' for v in p] for p in x1])

In [281]:
def color(x):
    if x == 'ch':
        return '#FF8DA1'
    elif x == 'sv':
        return '#99F4D2'
    else:
        return ''

In [328]:
def calculateW_arr(r,u,c):
    
    w = [[0 for _ in range(t_max)] for _ in range(n + 1)]
    x = [[False for _ in range(t_max)] for _ in range(n + 1)]
    
    for i in reversed(range(n + 1)):
          
        
        for t in range(i):
            if i == n :
                save_profit = r[i][t] - u[i][t]
                change_profit = r[i][0] - u[i][0] - c[i][t]
            else:
                save_profit = r[i][t] - u[i][t] + w[i + 1][t + 1]
                change_profit = r[i][0] - u[i][0] - c[i][t] + w[i + 1][1]

            if save_profit >= change_profit:
                w[i][t] = save_profit
                x[i][t] = "sv"
            else:
                w[i][t] = change_profit
                x[i][t] = "ch"
        
        if i == n :
            save_profit = r[i][i+2] - u[i][i+2]
            change_profit = r[i][0] - u[i][0] - c[i][i+2]
        else:
            save_profit = r[i][i+2] - u[i][i+2] + w[i + 1][i + 3]
            change_profit = r[i][0] - u[i][0] - c[i][i+2] + w[i + 1][1]

        if save_profit >= change_profit:
            w[i][i+2] = save_profit
            x[i][i+2] = "sv"
        else:
            w[i][i+2] = change_profit
            x[i][i+2] = "ch"  
    return w,x


In [300]:
def showStrategy(x):
    res = []
    k = 3
    for i in range(1,len(x)):
        
        if x[i][k] == 'ch':
            st = str(i) + '-й год: Следует заменить оборудование'
            k = 0
        elif x[i][k] == 'sv':
            st = str(i) + '-й год: Следует оставить текущее оборудование'
            k = k + 1
        
        res.append(st)    
    print("Оптимальная стратегия:\n")
    for s in res:
        print(s)
    return res

In [308]:
def saveStrategy(strat, file_name):
    outF = open(file_name, "w")
    for line in strat:
        outF.write(line)
        outF.write("\n")
    outF.close()

In [330]:
n = int(input("Введите длину расчетного периода\n"))
path = input("Введите путь к исходному файлу\n")
# n = 9
# path = 'data.xml'
path_to_strat = "strategy_1.txt"

machines = retrieveData(path)
t_max = findMax_t(machines)

r_arr = retrieveDataValues('r', machines, n, t_max)
print("\nПодсчитаем значение дохода от машины возраста t для года i")
print_table(r_arr,range(n + 1),range(t_max))

u_arr = retrieveDataValues('u', machines, n, t_max)
print("\nПодсчитаем значение затрат на содержание машины возраста t для года i")
u_df = pd.DataFrame(u_arr)
print_table(u_arr,range(n + 1),range(t_max))

c_arr = retrieveDataValues('c', machines, n, t_max)
print("\nПодсчитаем значение стоимости замены машины возраста t для года i")
c_df = pd.DataFrame(c_arr)
print_table(c_arr,range(n + 1),range(t_max))

w, x = calculateW_arr(r_arr,u_arr,c_arr)
print("\nМаксимальная прибыль составляет: " + str(w[1][3]) + " ед.\n")
        
w1, x1 = w[1:], x[1:]
strat = showStrategy(x)
saveStrategy(strat, path_to_strat)
print("\nПодсчитаем значение Fi(t)")    
df = pd.DataFrame(w1)
df1 = df.style.apply(add_color_lable, axis=None)
df1


Введите длину расчетного периода
9
Введите путь к исходному файлу
data.xml

Подсчитаем значение дохода от машины возраста t для года i
     0    1    2    3    4   5   6   7   8   9  10  11  12
0   -1   -1    0   -1   -1  -1  -1  -1  -1  -1  -1  -1  -1
1   90   -1   -1   60   -1  -1  -1  -1  -1  -1  -1  -1  -1
2  100   85   -1   -1   60  -1  -1  -1  -1  -1  -1  -1  -1
3  110   90   80   -1   -1  50  -1  -1  -1  -1  -1  -1  -1
4  115  105   80   75   -1  -1  50  -1  -1  -1  -1  -1  -1
5  120  110  100   75   70  -1  -1  50  -1  -1  -1  -1  -1
6  125  115  100   95   70  70  -1  -1  40  -1  -1  -1  -1
7  135  120  115   90   90  65  70  -1  -1  40  -1  -1  -1
8  140  125  110  110   80  80  65  60  -1  -1  40  -1  -1
9  150  135  110  105  105  70  70  65  60  -1  -1  30  -1

Подсчитаем значение затрат на содержание машины возраста t для года i
     0    1    2    3    4   5   6   7   8   9  10  11  12
0   -1   -1    0   -1   -1  -1  -1  -1  -1  -1  -1  -1  -1
1   90   -1   -1   60   -1 

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,480,0,0,240,0,0,0,0,0,0,0,0,0
1,460,410,0,0,235,0,0,0,0,0,0,0,0
2,500,375,345,0,0,230,0,0,0,0,0,0,0
3,445,405,305,290,0,0,205,0,0,0,0,0,0
4,495,345,315,245,240,0,0,215,0,0,0,0,0
5,415,385,250,235,160,155,0,0,125,0,0,0,0
6,340,300,280,170,160,100,75,0,0,50,0,0,0
7,260,215,190,180,100,95,65,35,0,0,-40,0,0
8,145,125,100,90,85,45,40,30,15,0,0,-35,0
